In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.nn.functional as F
import pickle
import itertools
import tqdm
# import hypergrad as hg
from mlmodel import *
import utils
import numpy as np
from sparsemax import Sparsemax
from argparse import ArgumentParser
from ZSTL_GPU import ZSTL
import pandas as pd
import itertools

In [2]:
net = FuncRecursiveNet([
    FLinearLayer(1, True)
])
shape_record = {0: [(1, 2049)]}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
kb_size = 10
train_size = 140

In [4]:
#filename = '../ZSTL_Data/Animals_with_Attributes2/splitedTask/task_train_data_standard.pickle'
filename = '../ZSTL_Data/CUB_200_2011/CUB_200_2011/splitedTask/task_train_data_standard.pickle'
with open(filename, 'rb') as f:
    dataset = pickle.load(f)
print(len(dataset), len(list(dataset.keys())))
print(type(dataset))

total_len = len(dataset)
test_size = total_len - kb_size - train_size


200 200
<class 'dict'>


In [5]:
#check stored data correctness
def check_attr_correct(num_task, dataset, task_attr_byID, model):
    task_train_data = {}
    task_test_data = {}
    task_val_data = {}
    for t in range(num_task):
        cur_attr = dataset[t+1][0]
        cur_param = utils.toTensor(dataset[t+1][1])
        cur_x = utils.toTensor(dataset[t+1][2])
        cur_y = dataset[t+1][3]
        print((cur_attr==task_attr_byID[t+1]).all())


        reshaped_w = utils.reshape_w(cur_param, shape_record)
        pred = model(reshaped_w, cur_x)
        pred = torch.sigmoid(pred)
        pred[pred>=0.5] = torch.ones_like(pred[pred>=0.5])
        pred[pred<0.5] = torch.zeros_like(pred[pred<0.5])
        print(np.sum(utils.toNumpy(pred)==cur_y)/100)



In [6]:
path_attributes = '../ZSTL_Data/Animals_with_Attributes2/predicate-matrix-continuous.txt'
def gen_attr(path_attributes):
    data_attributes = pd.read_csv(path_attributes, sep="\n", header=None)
    print(data_attributes)
    lst = []
    task_attr_byID = {}
    i = 1
    for r in data_attributes.iterrows():
        # print(len(r), )
        s = r[1].to_numpy()
        # print(len(s[0].split()), s[0].split())
        print(len(s[0].split()))
        task_attr_byID[i] = np.array([float(a) for a in s[0].split()])
        print(task_attr_byID[i].shape)
        i += 1
    return task_attr_byID

#task_attr_byID = gen_attr(path_attributes)
#check_attr_correct(total_len, dataset, task_attr_byID, net)


In [7]:
np.random.seed(666)
indx = [ x+1 for x in range(len(dataset))]
support_indx = list(np.random.choice(indx, size=kb_size, replace=False))
print(len(support_indx))
temp = [x for x in indx if x not in support_indx]
train_indx = list(np.random.choice(temp, size=train_size, replace=False))
temp = [x for x in temp if x not in train_indx]
print(len(train_indx))
test_indx = temp
print(len(test_indx))


support_data = utils.Dataset([dataset[d] for d in support_indx])
train_data = utils.Dataset([dataset[d] for d in train_indx])
test_data = utils.Dataset([dataset[d] for d in test_indx])
support_loader = DataLoader(support_data, batch_size=kb_size, shuffle=False)
train_loader = DataLoader(train_data, batch_size=train_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=test_size, shuffle=True)

10
140
50


In [8]:
support_a, support_w, support_x, support_y = next(iter(support_loader))
support_a, support_w, support_x, support_y = support_a.float(), support_w.float(), support_x.float(), support_y.float()
support_a = support_a.squeeze().t()
support_w = support_w.squeeze().t()
dm = support_a.size()[0]
print(dm)
d  = support_w.size()[0]
print(d)

param_dict = {}
param_dict['rho'] = 1e-5
param_dict['mu'] = 1e-5
param_dict['loss'] = 'binary class'
param_dict['outer lr'] = 1e-3
param_dict['align lr'] = 1e-3
param_dict['dm'] = dm
param_dict['d'] = d
param_dict['model_shape'] = shape_record
param_dict['atten_activation'] = 'Sparsemax'
print(param_dict)


ZSTL_model = ZSTL(support_w, support_a, support_x, net, param_dict, device)
ZSTL_model.train(train_loader, test_loader, max_iter=1500)


312
2049
{'rho': 1e-05, 'mu': 1e-05, 'loss': 'binary class', 'outer lr': 0.001, 'align lr': 0.001, 'dm': 312, 'd': 2049, 'model_shape': {0: [(1, 2049)]}, 'atten_activation': 'Sparsemax'}
init mean test metric 0.5932499999999999; align loss 0.15442827343940735
1/1500 o_loss 0.6687120677437633; m train metric 0.5103571428571431; m test metric 0.5132499999999999; align loss  0.14869605004787445
100/1500 o_loss 0.6454023142644604; m train metric 0.6550892857142858; m test metric 0.63475; align loss  0.13971823453903198
200/1500 o_loss 0.6397499790082553; m train metric 0.6656250000000002; m test metric 0.6459999999999999; align loss  0.1282854825258255
300/1500 o_loss 0.6326063864731363; m train metric 0.6772321428571427; m test metric 0.64575; align loss  0.11754855513572693
400/1500 o_loss 0.6250829269776919; m train metric 0.6885714285714284; m test metric 0.64475; align loss  0.1027536690235138
500/1500 o_loss 0.6156067197011518; m train metric 0.7030357142857143; m test metric 0.6465;

KeyboardInterrupt: 

In [ ]:



indx = train_indx
val_indx = list(np.random.choice(indx, size=20, replace=False))
print(len(val_indx))
train_indx_splited = [x for x in indx if x not in val_indx]
print(len(train_indx_splited))

val_data = utils.Dataset([dataset[d] for d in val_indx])
train_splited_data = utils.Dataset([dataset[d] for d in train_indx_splited])
val_loader = DataLoader(val_data, batch_size=len(val_indx), shuffle=True)
train_splited_loader = DataLoader(train_splited_data, batch_size=len(train_indx_splited), shuffle=True)

best_hp = utils.hp_select_binClass(train_splited_loader, val_loader, support_loader, d, dm, net, shape_record)
print('best_hp ', best_hp)

In [10]:


param_dict = {}
param_dict['rho'] = best_hp['rho']
param_dict['mu'] = best_hp['mu']
param_dict['loss'] = 'binary class'
param_dict['outer lr'] = 1e-3
param_dict['align lr'] = 1e-4
param_dict['dm'] = dm
param_dict['d'] = d
param_dict['model_shape'] = shape_record
param_dict['atten_activation'] = 'Sparsemax'
print(param_dict)


ZSTL_model = ZSTL(support_w, support_a, support_x, net, param_dict)
ZSTL_model.train(train_loader, test_loader, max_iter=1800)


2049 312
